<a href="https://colab.research.google.com/github/hectorcamachoz/Proyecto_2_Inteligenci_Artificial/blob/main/Proyecto2_594557.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto 2. Inteligencia Artificial

En este proyecto se utilizara la base de datos proporcionada por Kaggle especificamente de la competencia "Spaceship Titanic". El objetivo de este proyecto es realizar algunos de los multiples modelos vistos en clase, como LDA, QDA, Regresion Logistica, Árboles de decisión, bagging, random forests, boosting, y seleccionar cual es el mejor el resultado que se obtenga se mandara a la competencia de Kaggle. Nuestra variable de salida sera la variable Transported que tiene como valores True o False.


La descripcion de cada variable se encuentra en el archivo: 'descripcion_vars_kaggle.txt'


**1.** Primero se importara la base de datos train.csv, sera analizada con diferentes funciones para observar si existen huecos, si se deben realizar variables dummy, y de esta manera realizar una limpia.


In [63]:
import pandas as pd

df = pd.read_csv('train.csv')
print(df.head())

df.info()

print('\nHuecos: ')
df.isnull().sum()

  PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0001_01     Europa     False  B/0/P  TRAPPIST-1e  39.0  False   
1     0002_01      Earth     False  F/0/S  TRAPPIST-1e  24.0  False   
2     0003_01     Europa     False  A/0/S  TRAPPIST-1e  58.0   True   
3     0003_02     Europa     False  A/0/S  TRAPPIST-1e  33.0  False   
4     0004_01      Earth     False  F/1/S  TRAPPIST-1e  16.0  False   

   RoomService  FoodCourt  ShoppingMall     Spa  VRDeck               Name  \
0          0.0        0.0           0.0     0.0     0.0    Maham Ofracculy   
1        109.0        9.0          25.0   549.0    44.0       Juanna Vines   
2         43.0     3576.0           0.0  6715.0    49.0      Altark Susent   
3          0.0     1283.0         371.0  3329.0   193.0       Solam Susent   
4        303.0       70.0         151.0   565.0     2.0  Willy Santantines   

   Transported  
0        False  
1         True  
2        False  
3        False  
4         True  
<c

,0
PassengerId,0
HomePlanet,201
CryoSleep,217
Cabin,199
Destination,182
Age,179
VIP,203
RoomService,181
FoodCourt,183
ShoppingMall,208


Se puede observar que la muchas variables son de tipo object, por lo tanto para primero utilizar regresion logistica, es necesario convertir estas variables a variables dummy.

Primero se eliminaran las variables PassengerId y Name, ya que esas no nos serviran para el entrenamiento del modelo.

In [64]:
df = df.drop(['PassengerId', 'Name'], axis=1)

En la tabla de huecos en el primer apartado, se puede observar que existen mulitples en variables tipo float, por lo tanto, los huecos en esas variables se sustituiran por 0.

In [65]:
df[['VIP', 'CryoSleep', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']] = df[['VIP', 'CryoSleep', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].fillna(value=0)
df.isnull().sum().sort_values(ascending=False)

,0
HomePlanet,201
Cabin,199
Destination,182
RoomService,181
Age,179
CryoSleep,0
VIP,0
FoodCourt,0
ShoppingMall,0
Spa,0


Para evitar problemas con la regresion logistica, necesitamos convertir los datos de object, y los booleanos a variables numericas.

In [66]:
df['Transported'] = df['Transported'].astype(int)
df['CryoSleep'] = df['CryoSleep'].astype(int)
df['VIP'] = df['VIP'].astype(int)

df.head(5)

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,0,B/0/P,TRAPPIST-1e,39.0,0,0.0,0.0,0.0,0.0,0.0,0
1,Earth,0,F/0/S,TRAPPIST-1e,24.0,0,109.0,9.0,25.0,549.0,44.0,1
2,Europa,0,A/0/S,TRAPPIST-1e,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0
3,Europa,0,A/0/S,TRAPPIST-1e,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0
4,Earth,0,F/1/S,TRAPPIST-1e,16.0,0,303.0,70.0,151.0,565.0,2.0,1


La variable Cabin, esta compuesta por tres datos, Deck/Cabin number/Side. Para facilitar esta variable se dividira en tres columnas. Este codigo es una combinacion de algunos competidores uno llamado Gusthema y otro llamado Eunice Cleofas

In [67]:

df[["Deck", "Cabin_num", "Side"]] = df["Cabin"].str.split("/", expand=True)
df = df.drop(['Cabin'], axis = 1)
df[['Cabin_num']] = df[['Cabin_num']].fillna(df['Cabin_num'].mode())
df[['Deck']] = df[['Deck']].fillna(df['Deck'].mode())
df[['Side']] = df[['Side']].fillna(df['Side'].mode())
df.head(5)

df.isnull().sum().sort_values(ascending=False)

,0
HomePlanet,201
Cabin_num,199
Side,199
Deck,199
Destination,182
RoomService,181
Age,179
CryoSleep,0
VIP,0
FoodCourt,0


Para terminar con la limpieza del dataset, se convertiran las variables HomePlanet y Destination a variables de 0 y 1.

In [68]:
df = pd.get_dummies(df,columns= ['HomePlanet', 'Destination'], drop_first = True)
df.head()

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Cabin_num,Side,HomePlanet_Europa,HomePlanet_Mars,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0,39.0,0,0.0,0.0,0.0,0.0,0.0,0,B,0,P,True,False,False,True
1,0,24.0,0,109.0,9.0,25.0,549.0,44.0,1,F,0,S,False,False,False,True
2,0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0,A,0,S,True,False,False,True
3,0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0,A,0,S,True,False,False,True
4,0,16.0,0,303.0,70.0,151.0,565.0,2.0,1,F,1,S,False,False,False,True


In [69]:
df['HomePlanet_Europa'] = df['HomePlanet_Europa'].astype(int)
df['HomePlanet_Mars'] = df['HomePlanet_Mars'].astype(int)
df['Destination_TRAPPIST-1e'] = df['Destination_TRAPPIST-1e'].astype(int)
df['Destination_PSO J318.5-22'] = df['Destination_PSO J318.5-22'].astype(int)
df.head()

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Cabin_num,Side,HomePlanet_Europa,HomePlanet_Mars,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0,39.0,0,0.0,0.0,0.0,0.0,0.0,0,B,0,P,1,0,0,1
1,0,24.0,0,109.0,9.0,25.0,549.0,44.0,1,F,0,S,0,0,0,1
2,0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0,A,0,S,1,0,0,1
3,0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0,A,0,S,1,0,0,1
4,0,16.0,0,303.0,70.0,151.0,565.0,2.0,1,F,1,S,0,0,0,1


In [70]:
print(df.isnull().sum())

CryoSleep                      0
Age                          179
VIP                            0
RoomService                  181
FoodCourt                      0
ShoppingMall                   0
Spa                            0
VRDeck                         0
Transported                    0
Deck                         199
Cabin_num                    199
Side                         199
HomePlanet_Europa              0
HomePlanet_Mars                0
Destination_PSO J318.5-22      0
Destination_TRAPPIST-1e        0
dtype: int64


Todavia se tienen huecos en la base de datos por lo tanto, en la variable Age, se establecera el promedio de todos los datos y ese se pondra en los huecos. En la variable de room service se realizara lo mismo.

In [71]:
df['Age'] = df['Age'].fillna(df['Age'].mean())
df['RoomService'] = df['RoomService'].fillna(df['RoomService'].mean())
df.isnull().sum().sort_values(ascending=False)

,0
Cabin_num,199
Side,199
Deck,199
CryoSleep,0
FoodCourt,0
Age,0
VIP,0
RoomService,0
VRDeck,0
Spa,0


2. Se realizara un modelo de regresion logistica